# Module 8: GAN 與 Diffusion 模型

## 學習目標
- 理解 GAN 的對抗訓練概念
- 實現 DCGAN 生成圖像
- 了解 Diffusion 模型的原理
- 實現簡單的 DDPM

## 為什麼要學這些模型？

GAN 和 Diffusion 是目前最強大的圖像生成模型：
- **GAN (2014)**：開創性的對抗訓練，生成逼真圖像
- **Diffusion (2020+)**：更穩定、更高品質，Stable Diffusion 等的基礎

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import numpy as np
from tqdm.auto import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用設備: {device}")

# 固定隨機種子
torch.manual_seed(42)
np.random.seed(42)

---
## Part 1: GAN 基礎概念

### 直觀理解

GAN 就像是「偽鈔製造者 vs 警察」的對抗遊戲：

```
┌─────────────────────────────────────────────────────┐
│                                                     │
│   噪音 z ──► [生成器 G] ──► 假圖像                   │
│               偽鈔製造者      │                     │
│                              ▼                     │
│   真實圖像 ──────────────► [判別器 D] ──► 真/假？    │
│                              警察                   │
│                                                     │
│   目標：                                            │
│   - G 想騙過 D（生成越來越逼真的圖像）              │
│   - D 想正確分辨真假                                │
│   - 最終 G 生成的圖像逼真到 D 無法分辨              │
│                                                     │
└─────────────────────────────────────────────────────┘
```

### 數學表述

GAN 的目標函數（最小-最大遊戲）：

$$\min_G \max_D V(D, G) = \mathbb{E}_{x \sim p_{data}}[\log D(x)] + \mathbb{E}_{z \sim p_z}[\log(1 - D(G(z)))]$$

- 判別器 D 想最大化這個值（正確分類真假）
- 生成器 G 想最小化這個值（騙過判別器）

### 載入資料

In [ ]:
# 使用 MNIST 進行示範
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # 歸一化到 [-1, 1]
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

print(f"訓練集大小: {len(train_dataset)}")

# 顯示樣本
def show_images(images, nrow=8, title=""):
    """顯示圖像網格"""
    grid = make_grid(images, nrow=nrow, normalize=True, value_range=(-1, 1))
    plt.figure(figsize=(12, 6))
    plt.imshow(grid.permute(1, 2, 0).cpu().numpy(), cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

sample_images, _ = next(iter(train_loader))
show_images(sample_images[:32], title="真實 MNIST 樣本")

---
## Part 2: 簡單 GAN 實現

In [ ]:
class Generator(nn.Module):
    """
    生成器：從隨機噪音生成圖像
    
    輸入：latent_dim 維的隨機向量
    輸出：28x28 的圖像
    """
    
    def __init__(self, latent_dim=100, img_dim=784):
        super().__init__()
        
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(256),
            
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(512),
            
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(1024),
            
            nn.Linear(1024, img_dim),
            nn.Tanh(),  # 輸出在 [-1, 1]
        )
        
    def forward(self, z):
        return self.model(z)

class Discriminator(nn.Module):
    """
    判別器：判斷圖像是真是假
    
    輸入：28x28 的圖像
    輸出：真假機率（0-1）
    """
    
    def __init__(self, img_dim=784):
        super().__init__()
        
        self.model = nn.Sequential(
            nn.Linear(img_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(256, 1),
            nn.Sigmoid(),  # 輸出機率
        )
        
    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        return self.model(img_flat)

# 測試
latent_dim = 100
G = Generator(latent_dim)
D = Discriminator()

z = torch.randn(4, latent_dim)
fake_imgs = G(z)
validity = D(fake_imgs.view(-1, 1, 28, 28))

print("GAN 架構測試：")
print(f"噪音形狀: {z.shape}")
print(f"生成圖像形狀: {fake_imgs.shape}")
print(f"判別器輸出: {validity.squeeze()}")

In [ ]:
def train_gan(generator, discriminator, train_loader, epochs=50, latent_dim=100, lr=0.0002):
    """
    訓練 GAN
    
    關鍵步驟：
    1. 訓練判別器：區分真假
    2. 訓練生成器：騙過判別器
    """
    
    generator = generator.to(device)
    discriminator = discriminator.to(device)
    
    # 使用 Adam 優化器
    opt_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    opt_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
    
    criterion = nn.BCELoss()
    
    # 用於生成樣本的固定噪音
    fixed_noise = torch.randn(64, latent_dim).to(device)
    
    history = {'d_loss': [], 'g_loss': []}
    
    for epoch in range(epochs):
        g_losses = []
        d_losses = []
        
        for real_imgs, _ in train_loader:
            batch_size = real_imgs.size(0)
            real_imgs = real_imgs.to(device)
            
            # 真假標籤
            real_labels = torch.ones(batch_size, 1).to(device)
            fake_labels = torch.zeros(batch_size, 1).to(device)
            
            # ========== 訓練判別器 ==========
            opt_D.zero_grad()
            
            # 真實圖像的損失
            real_validity = discriminator(real_imgs)
            d_loss_real = criterion(real_validity, real_labels)
            
            # 生成假圖像
            z = torch.randn(batch_size, latent_dim).to(device)
            fake_imgs = generator(z).view(-1, 1, 28, 28)
            
            # 假圖像的損失（detach 避免梯度傳到生成器）
            fake_validity = discriminator(fake_imgs.detach())
            d_loss_fake = criterion(fake_validity, fake_labels)
            
            d_loss = (d_loss_real + d_loss_fake) / 2
            d_loss.backward()
            opt_D.step()
            
            # ========== 訓練生成器 ==========
            opt_G.zero_grad()
            
            # 生成器想讓判別器認為假圖像是真的
            fake_validity = discriminator(fake_imgs)
            g_loss = criterion(fake_validity, real_labels)  # 注意：用 real_labels
            
            g_loss.backward()
            opt_G.step()
            
            d_losses.append(d_loss.item())
            g_losses.append(g_loss.item())
        
        avg_d_loss = np.mean(d_losses)
        avg_g_loss = np.mean(g_losses)
        history['d_loss'].append(avg_d_loss)
        history['g_loss'].append(avg_g_loss)
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}")
            print(f"  D Loss: {avg_d_loss:.4f}, G Loss: {avg_g_loss:.4f}")
            
            # 生成樣本
            generator.eval()
            with torch.no_grad():
                samples = generator(fixed_noise).view(-1, 1, 28, 28)
            show_images(samples, nrow=8, title=f"Epoch {epoch+1}")
            generator.train()
    
    return history

# 訓練
G = Generator(latent_dim=100).to(device)
D = Discriminator().to(device)

gan_history = train_gan(G, D, train_loader, epochs=50)

In [ ]:
# 繪製訓練曲線
plt.figure(figsize=(10, 4))
plt.plot(gan_history['d_loss'], label='Discriminator Loss')
plt.plot(gan_history['g_loss'], label='Generator Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('GAN 訓練曲線')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---
## Part 3: DCGAN (Deep Convolutional GAN)

DCGAN 使用卷積/轉置卷積，生成更高品質的圖像。

In [ ]:
class DCGenerator(nn.Module):
    """
    DCGAN 生成器
    
    使用轉置卷積逐步上採樣
    """
    
    def __init__(self, latent_dim=100, channels=1, features=64):
        super().__init__()
        
        self.latent_dim = latent_dim
        
        # 輸入: [B, latent_dim]
        # 輸出: [B, channels, 28, 28]
        
        self.model = nn.Sequential(
            # 投影並重塑: [B, latent_dim] -> [B, features*4, 7, 7]
            nn.Linear(latent_dim, features * 4 * 7 * 7),
            nn.BatchNorm1d(features * 4 * 7 * 7),
            nn.ReLU(True),
        )
        
        self.conv_blocks = nn.Sequential(
            # [B, features*4, 7, 7] -> [B, features*2, 14, 14]
            nn.ConvTranspose2d(features * 4, features * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features * 2),
            nn.ReLU(True),
            
            # [B, features*2, 14, 14] -> [B, channels, 28, 28]
            nn.ConvTranspose2d(features * 2, channels, 4, 2, 1, bias=False),
            nn.Tanh(),
        )
        
        self.features = features
        
    def forward(self, z):
        x = self.model(z)
        x = x.view(x.size(0), self.features * 4, 7, 7)
        return self.conv_blocks(x)

class DCDiscriminator(nn.Module):
    """
    DCGAN 判別器
    
    使用步進卷積逐步下採樣
    """
    
    def __init__(self, channels=1, features=64):
        super().__init__()
        
        self.model = nn.Sequential(
            # [B, channels, 28, 28] -> [B, features, 14, 14]
            nn.Conv2d(channels, features, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            # [B, features, 14, 14] -> [B, features*2, 7, 7]
            nn.Conv2d(features, features * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(features * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            # [B, features*2, 7, 7] -> [B, 1]
            nn.Flatten(),
            nn.Linear(features * 2 * 7 * 7, 1),
            nn.Sigmoid(),
        )
        
    def forward(self, img):
        return self.model(img)

# 測試
dc_G = DCGenerator(latent_dim=100)
dc_D = DCDiscriminator()

z = torch.randn(4, 100)
fake = dc_G(z)
validity = dc_D(fake)

print("DCGAN 架構測試：")
print(f"生成圖像形狀: {fake.shape}")
print(f"判別器輸出: {validity.squeeze()}")

In [ ]:
def train_dcgan(generator, discriminator, train_loader, epochs=30, latent_dim=100, lr=0.0002):
    """
    訓練 DCGAN
    """
    
    generator = generator.to(device)
    discriminator = discriminator.to(device)
    
    opt_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    opt_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
    
    criterion = nn.BCELoss()
    fixed_noise = torch.randn(64, latent_dim).to(device)
    
    history = {'d_loss': [], 'g_loss': []}
    
    for epoch in range(epochs):
        g_losses = []
        d_losses = []
        
        for real_imgs, _ in train_loader:
            batch_size = real_imgs.size(0)
            real_imgs = real_imgs.to(device)
            
            real_labels = torch.ones(batch_size, 1).to(device)
            fake_labels = torch.zeros(batch_size, 1).to(device)
            
            # ========== 訓練判別器 ==========
            opt_D.zero_grad()
            
            d_loss_real = criterion(discriminator(real_imgs), real_labels)
            
            z = torch.randn(batch_size, latent_dim).to(device)
            fake_imgs = generator(z)
            d_loss_fake = criterion(discriminator(fake_imgs.detach()), fake_labels)
            
            d_loss = (d_loss_real + d_loss_fake) / 2
            d_loss.backward()
            opt_D.step()
            
            # ========== 訓練生成器 ==========
            opt_G.zero_grad()
            
            g_loss = criterion(discriminator(fake_imgs), real_labels)
            g_loss.backward()
            opt_G.step()
            
            d_losses.append(d_loss.item())
            g_losses.append(g_loss.item())
        
        history['d_loss'].append(np.mean(d_losses))
        history['g_loss'].append(np.mean(g_losses))
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, D Loss: {history['d_loss'][-1]:.4f}, G Loss: {history['g_loss'][-1]:.4f}")
            
            generator.eval()
            with torch.no_grad():
                samples = generator(fixed_noise)
            show_images(samples, nrow=8, title=f"DCGAN Epoch {epoch+1}")
            generator.train()
    
    return history

# 訓練 DCGAN
dc_G = DCGenerator(latent_dim=100).to(device)
dc_D = DCDiscriminator().to(device)

dcgan_history = train_dcgan(dc_G, dc_D, train_loader, epochs=30)

### 潛在空間探索

In [ ]:
def latent_space_interpolation_gan(generator, latent_dim=100, n_steps=10):
    """
    GAN 潛在空間插值
    """
    generator.eval()
    
    # 兩個隨機起點
    z1 = torch.randn(1, latent_dim).to(device)
    z2 = torch.randn(1, latent_dim).to(device)
    
    interpolations = []
    
    with torch.no_grad():
        for alpha in np.linspace(0, 1, n_steps):
            z = (1 - alpha) * z1 + alpha * z2
            img = generator(z)
            interpolations.append(img)
    
    interpolations = torch.cat(interpolations, dim=0)
    show_images(interpolations, nrow=n_steps, title="GAN 潛在空間插值")

latent_space_interpolation_gan(dc_G)

---
## Part 4: Diffusion 模型基礎

### 直觀理解

Diffusion 模型的想法很簡單：

```
前向過程（加噪）：逐步向圖像添加高斯噪音
x₀ ──► x₁ ──► x₂ ──► ... ──► xT (純噪音)
原圖    +噪    +噪           純噪音

反向過程（去噪）：學習逐步去除噪音
xT ──► xT-1 ──► ... ──► x₁ ──► x₀
噪音   -噪      -噪      -噪    清晰圖像

訓練目標：預測在每一步添加的噪音
```

### 數學表述

**前向過程**（固定的）：
$$q(x_t | x_{t-1}) = \mathcal{N}(x_t; \sqrt{1-\beta_t} x_{t-1}, \beta_t I)$$

**反向過程**（學習的）：
$$p_\theta(x_{t-1} | x_t) = \mathcal{N}(x_{t-1}; \mu_\theta(x_t, t), \Sigma_\theta(x_t, t))$$

**簡化的訓練目標**：
$$L = \mathbb{E}_{t, x_0, \epsilon}[||\epsilon - \epsilon_\theta(x_t, t)||^2]$$

即：預測添加的噪音

In [ ]:
class DiffusionScheduler:
    """
    Diffusion 過程的時間表
    
    定義了在每個時間步的噪音級別
    """
    
    def __init__(self, num_timesteps=1000, beta_start=1e-4, beta_end=0.02):
        self.num_timesteps = num_timesteps
        
        # 線性調度 beta
        self.betas = torch.linspace(beta_start, beta_end, num_timesteps)
        
        # 計算累積乘積
        self.alphas = 1.0 - self.betas
        self.alpha_cumprod = torch.cumprod(self.alphas, dim=0)
        self.alpha_cumprod_prev = F.pad(self.alpha_cumprod[:-1], (1, 0), value=1.0)
        
        # 預計算常用值
        self.sqrt_alpha_cumprod = torch.sqrt(self.alpha_cumprod)
        self.sqrt_one_minus_alpha_cumprod = torch.sqrt(1.0 - self.alpha_cumprod)
        
    def add_noise(self, x_0, t, noise=None):
        """
        前向過程：給圖像添加噪音
        
        x_t = sqrt(alpha_cumprod_t) * x_0 + sqrt(1 - alpha_cumprod_t) * noise
        """
        if noise is None:
            noise = torch.randn_like(x_0)
        
        sqrt_alpha = self.sqrt_alpha_cumprod[t].view(-1, 1, 1, 1).to(x_0.device)
        sqrt_one_minus_alpha = self.sqrt_one_minus_alpha_cumprod[t].view(-1, 1, 1, 1).to(x_0.device)
        
        x_t = sqrt_alpha * x_0 + sqrt_one_minus_alpha * noise
        return x_t, noise

# 可視化前向過程
scheduler = DiffusionScheduler(num_timesteps=1000)

# 取一張圖片
sample_img = sample_images[0:1]  # [1, 1, 28, 28]

# 在不同時間步添加噪音
timesteps = [0, 100, 250, 500, 750, 999]
noisy_images = []

for t in timesteps:
    t_tensor = torch.tensor([t])
    noisy, _ = scheduler.add_noise(sample_img, t_tensor)
    noisy_images.append(noisy)

noisy_images = torch.cat(noisy_images, dim=0)

fig, axes = plt.subplots(1, len(timesteps), figsize=(15, 3))
for i, t in enumerate(timesteps):
    axes[i].imshow(noisy_images[i].squeeze().numpy(), cmap='gray')
    axes[i].set_title(f't = {t}')
    axes[i].axis('off')

plt.suptitle('Diffusion 前向過程：逐步添加噪音')
plt.tight_layout()
plt.show()

### 簡化的 U-Net 噪音預測器

In [ ]:
class SinusoidalPositionEmbedding(nn.Module):
    """
    時間步的正弦位置編碼
    
    將時間步 t 編碼為高維向量，讓模型知道當前是第幾步
    """
    
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
        
    def forward(self, t):
        device = t.device
        half_dim = self.dim // 2
        
        embeddings = np.log(10000) / (half_dim - 1)
        embeddings = torch.exp(torch.arange(half_dim, device=device) * -embeddings)
        embeddings = t[:, None] * embeddings[None, :]
        embeddings = torch.cat([torch.sin(embeddings), torch.cos(embeddings)], dim=-1)
        
        return embeddings

class SimpleUNet(nn.Module):
    """
    簡化版 U-Net 用於噪音預測
    
    對於 MNIST 這樣的簡單資料，不需要完整的 U-Net
    """
    
    def __init__(self, in_channels=1, time_dim=256, features=64):
        super().__init__()
        
        self.time_mlp = nn.Sequential(
            SinusoidalPositionEmbedding(time_dim),
            nn.Linear(time_dim, time_dim),
            nn.GELU(),
            nn.Linear(time_dim, time_dim),
        )
        
        # 下採樣路徑
        self.down1 = self._conv_block(in_channels, features)
        self.down2 = self._conv_block(features, features * 2)
        self.pool = nn.MaxPool2d(2)
        
        # 中間
        self.mid = self._conv_block(features * 2, features * 2)
        
        # 時間嵌入投影
        self.time_proj = nn.Linear(time_dim, features * 2)
        
        # 上採樣路徑
        self.up1 = nn.ConvTranspose2d(features * 2, features * 2, 2, 2)
        self.up_conv1 = self._conv_block(features * 4, features)  # 跳躍連接
        self.up2 = nn.ConvTranspose2d(features, features, 2, 2)
        self.up_conv2 = self._conv_block(features * 2, features)  # 跳躍連接
        
        # 輸出
        self.out = nn.Conv2d(features, in_channels, 1)
        
    def _conv_block(self, in_ch, out_ch):
        return nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.GroupNorm(8, out_ch),
            nn.GELU(),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.GroupNorm(8, out_ch),
            nn.GELU(),
        )
        
    def forward(self, x, t):
        # 時間嵌入
        t_emb = self.time_mlp(t.float())
        
        # 下採樣
        d1 = self.down1(x)                    # [B, 64, 28, 28]
        d2 = self.down2(self.pool(d1))        # [B, 128, 14, 14]
        
        # 中間（加入時間資訊）
        m = self.mid(self.pool(d2))           # [B, 128, 7, 7]
        t_proj = self.time_proj(t_emb)[:, :, None, None]  # [B, 128, 1, 1]
        m = m + t_proj
        
        # 上採樣
        u1 = self.up1(m)                      # [B, 128, 14, 14]
        u1 = torch.cat([u1, d2], dim=1)       # [B, 256, 14, 14]
        u1 = self.up_conv1(u1)                # [B, 64, 14, 14]
        
        u2 = self.up2(u1)                     # [B, 64, 28, 28]
        u2 = torch.cat([u2, d1], dim=1)       # [B, 128, 28, 28]
        u2 = self.up_conv2(u2)                # [B, 64, 28, 28]
        
        return self.out(u2)                   # [B, 1, 28, 28]

# 測試
unet = SimpleUNet()
x = torch.randn(4, 1, 28, 28)
t = torch.randint(0, 1000, (4,))
out = unet(x, t)
print(f"U-Net 輸入形狀: {x.shape}")
print(f"U-Net 輸出形狀: {out.shape}")

### 訓練 Diffusion 模型

In [ ]:
def train_diffusion(model, scheduler, train_loader, epochs=20, lr=1e-4):
    """
    訓練 Diffusion 模型
    
    訓練目標：預測添加到圖像的噪音
    """
    
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    history = {'loss': []}
    
    for epoch in range(epochs):
        model.train()
        losses = []
        
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
        
        for images, _ in pbar:
            images = images.to(device)
            batch_size = images.size(0)
            
            # 隨機選擇時間步
            t = torch.randint(0, scheduler.num_timesteps, (batch_size,)).to(device)
            
            # 添加噪音
            noise = torch.randn_like(images)
            noisy_images, _ = scheduler.add_noise(images, t.cpu(), noise)
            noisy_images = noisy_images.to(device)
            
            # 預測噪音
            predicted_noise = model(noisy_images, t)
            
            # 損失：預測噪音與實際噪音的 MSE
            loss = F.mse_loss(predicted_noise, noise)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            losses.append(loss.item())
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        avg_loss = np.mean(losses)
        history['loss'].append(avg_loss)
        print(f"Epoch {epoch+1}, Avg Loss: {avg_loss:.4f}")
    
    return history

# 訓練
diffusion_model = SimpleUNet().to(device)
scheduler = DiffusionScheduler(num_timesteps=1000)

diffusion_history = train_diffusion(diffusion_model, scheduler, train_loader, epochs=20)

### 從 Diffusion 模型生成樣本

In [ ]:
@torch.no_grad()
def sample_diffusion(model, scheduler, n_samples=16, img_size=28):
    """
    從 Diffusion 模型生成樣本
    
    反向過程：從純噪音逐步去噪
    """
    
    model.eval()
    
    # 從純噪音開始
    x = torch.randn(n_samples, 1, img_size, img_size).to(device)
    
    # 反向迭代
    for t in tqdm(reversed(range(scheduler.num_timesteps)), desc="Sampling", total=scheduler.num_timesteps):
        t_batch = torch.full((n_samples,), t, dtype=torch.long).to(device)
        
        # 預測噪音
        predicted_noise = model(x, t_batch)
        
        # 計算係數
        alpha = scheduler.alphas[t]
        alpha_cumprod = scheduler.alpha_cumprod[t]
        alpha_cumprod_prev = scheduler.alpha_cumprod_prev[t]
        beta = scheduler.betas[t]
        
        # 計算均值
        coef1 = 1 / torch.sqrt(alpha)
        coef2 = beta / torch.sqrt(1 - alpha_cumprod)
        
        mean = coef1 * (x - coef2 * predicted_noise)
        
        # 添加噪音（除了最後一步）
        if t > 0:
            noise = torch.randn_like(x)
            sigma = torch.sqrt(beta)
            x = mean + sigma * noise
        else:
            x = mean
    
    return x

# 生成樣本
samples = sample_diffusion(diffusion_model, scheduler, n_samples=64)
show_images(samples, nrow=8, title="Diffusion 模型生成的樣本")

In [ ]:
@torch.no_grad()
def visualize_generation_process(model, scheduler, n_steps_to_show=10):
    """
    可視化生成過程的中間步驟
    """
    
    model.eval()
    
    x = torch.randn(1, 1, 28, 28).to(device)
    
    # 選擇要顯示的時間步
    steps_to_show = np.linspace(scheduler.num_timesteps - 1, 0, n_steps_to_show, dtype=int)
    intermediate_images = []
    
    for t in reversed(range(scheduler.num_timesteps)):
        t_batch = torch.tensor([t]).to(device)
        
        predicted_noise = model(x, t_batch)
        
        alpha = scheduler.alphas[t]
        alpha_cumprod = scheduler.alpha_cumprod[t]
        beta = scheduler.betas[t]
        
        coef1 = 1 / torch.sqrt(alpha)
        coef2 = beta / torch.sqrt(1 - alpha_cumprod)
        
        mean = coef1 * (x - coef2 * predicted_noise)
        
        if t > 0:
            noise = torch.randn_like(x)
            sigma = torch.sqrt(beta)
            x = mean + sigma * noise
        else:
            x = mean
        
        if t in steps_to_show:
            intermediate_images.append((t, x.clone()))
    
    # 可視化
    fig, axes = plt.subplots(1, len(intermediate_images), figsize=(15, 2))
    
    for i, (t, img) in enumerate(intermediate_images):
        axes[i].imshow(img.cpu().squeeze(), cmap='gray')
        axes[i].set_title(f't = {t}')
        axes[i].axis('off')
    
    plt.suptitle('Diffusion 反向過程：從噪音到清晰圖像')
    plt.tight_layout()
    plt.show()

visualize_generation_process(diffusion_model, scheduler)

---
## 練習題

### 練習 1: 條件 GAN (CGAN)

**目標**: 實現條件 GAN，可以指定生成特定數字

In [ ]:
# 練習 1: 條件 GAN

class ConditionalGenerator(nn.Module):
    """
    條件生成器：輸入噪音 + 類別標籤
    """
    
    def __init__(self, latent_dim=100, num_classes=10, img_dim=784):
        super().__init__()
        
        # 類別嵌入
        self.label_embedding = nn.Embedding(num_classes, num_classes)
        
        self.model = nn.Sequential(
            nn.Linear(latent_dim + num_classes, 256),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(256),
            
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(512),
            
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(1024),
            
            nn.Linear(1024, img_dim),
            nn.Tanh(),
        )
        
    def forward(self, z, labels):
        label_emb = self.label_embedding(labels)
        x = torch.cat([z, label_emb], dim=1)
        return self.model(x)

class ConditionalDiscriminator(nn.Module):
    """
    條件判別器：輸入圖像 + 類別標籤
    """
    
    def __init__(self, num_classes=10, img_dim=784):
        super().__init__()
        
        self.label_embedding = nn.Embedding(num_classes, num_classes)
        
        self.model = nn.Sequential(
            nn.Linear(img_dim + num_classes, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )
        
    def forward(self, img, labels):
        img_flat = img.view(img.size(0), -1)
        label_emb = self.label_embedding(labels)
        x = torch.cat([img_flat, label_emb], dim=1)
        return self.model(x)

def train_cgan(generator, discriminator, train_loader, epochs=50, latent_dim=100, lr=0.0002):
    """
    訓練條件 GAN
    """
    
    generator = generator.to(device)
    discriminator = discriminator.to(device)
    
    opt_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0.5, 0.999))
    opt_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(0.5, 0.999))
    
    criterion = nn.BCELoss()
    
    for epoch in range(epochs):
        g_losses = []
        d_losses = []
        
        for real_imgs, labels in train_loader:
            batch_size = real_imgs.size(0)
            real_imgs = real_imgs.to(device)
            labels = labels.to(device)
            
            real_labels = torch.ones(batch_size, 1).to(device)
            fake_labels = torch.zeros(batch_size, 1).to(device)
            
            # 訓練判別器
            opt_D.zero_grad()
            
            d_loss_real = criterion(discriminator(real_imgs, labels), real_labels)
            
            z = torch.randn(batch_size, latent_dim).to(device)
            fake_imgs = generator(z, labels).view(-1, 1, 28, 28)
            d_loss_fake = criterion(discriminator(fake_imgs.detach(), labels), fake_labels)
            
            d_loss = (d_loss_real + d_loss_fake) / 2
            d_loss.backward()
            opt_D.step()
            
            # 訓練生成器
            opt_G.zero_grad()
            
            g_loss = criterion(discriminator(fake_imgs, labels), real_labels)
            g_loss.backward()
            opt_G.step()
            
            d_losses.append(d_loss.item())
            g_losses.append(g_loss.item())
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}, D Loss: {np.mean(d_losses):.4f}, G Loss: {np.mean(g_losses):.4f}")
    
    return generator, discriminator

# 訓練
cgan_G = ConditionalGenerator(latent_dim=100)
cgan_D = ConditionalDiscriminator()

cgan_G, cgan_D = train_cgan(cgan_G, cgan_D, train_loader, epochs=50)

In [ ]:
# 測試條件生成
def generate_conditional_samples_gan(generator, latent_dim=100, n_per_class=5):
    generator.eval()
    
    fig, axes = plt.subplots(10, n_per_class, figsize=(n_per_class * 1.5, 15))
    
    with torch.no_grad():
        for digit in range(10):
            z = torch.randn(n_per_class, latent_dim).to(device)
            labels = torch.full((n_per_class,), digit, dtype=torch.long).to(device)
            samples = generator(z, labels).view(-1, 28, 28)
            
            for i in range(n_per_class):
                axes[digit, i].imshow(samples[i].cpu(), cmap='gray')
                axes[digit, i].axis('off')
            
            axes[digit, 0].set_ylabel(str(digit), fontsize=14, rotation=0, labelpad=15)
    
    plt.suptitle('條件 GAN 生成結果')
    plt.tight_layout()
    plt.show()

generate_conditional_samples_gan(cgan_G, n_per_class=8)

### 練習 2: WGAN-GP

**目標**: 實現 Wasserstein GAN with Gradient Penalty，改善訓練穩定性

In [ ]:
# 練習 2: WGAN-GP

class WGANCritic(nn.Module):
    """
    WGAN 評論器（不是判別器）
    
    輸出不需要 Sigmoid，直接輸出分數
    """
    
    def __init__(self, img_dim=784):
        super().__init__()
        
        self.model = nn.Sequential(
            nn.Linear(img_dim, 512),
            nn.LeakyReLU(0.2),
            
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            
            nn.Linear(256, 1),
            # 無 Sigmoid！
        )
        
    def forward(self, img):
        return self.model(img.view(img.size(0), -1))

def compute_gradient_penalty(critic, real_imgs, fake_imgs):
    """
    計算梯度懲罰
    
    在真實和假圖像之間插值，計算梯度範數
    """
    batch_size = real_imgs.size(0)
    
    # 隨機插值係數
    alpha = torch.rand(batch_size, 1, 1, 1).to(real_imgs.device)
    
    # 插值圖像
    interpolated = (alpha * real_imgs + (1 - alpha) * fake_imgs).requires_grad_(True)
    
    # 計算評論器輸出
    d_interpolated = critic(interpolated)
    
    # 計算梯度
    gradients = torch.autograd.grad(
        outputs=d_interpolated,
        inputs=interpolated,
        grad_outputs=torch.ones_like(d_interpolated),
        create_graph=True,
        retain_graph=True,
    )[0]
    
    gradients = gradients.view(batch_size, -1)
    gradient_norm = gradients.norm(2, dim=1)
    gradient_penalty = ((gradient_norm - 1) ** 2).mean()
    
    return gradient_penalty

def train_wgan_gp(generator, critic, train_loader, epochs=50, latent_dim=100, 
                  lr=1e-4, n_critic=5, lambda_gp=10):
    """
    訓練 WGAN-GP
    
    關鍵改變：
    - 評論器訓練更多次
    - 使用 Wasserstein 損失
    - 添加梯度懲罰
    """
    
    generator = generator.to(device)
    critic = critic.to(device)
    
    opt_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(0, 0.9))
    opt_C = torch.optim.Adam(critic.parameters(), lr=lr, betas=(0, 0.9))
    
    fixed_noise = torch.randn(64, latent_dim).to(device)
    history = {'c_loss': [], 'g_loss': []}
    
    for epoch in range(epochs):
        c_losses = []
        g_losses = []
        
        for batch_idx, (real_imgs, _) in enumerate(train_loader):
            batch_size = real_imgs.size(0)
            real_imgs = real_imgs.to(device)
            
            # 訓練評論器（多次）
            for _ in range(n_critic):
                opt_C.zero_grad()
                
                # 生成假圖像
                z = torch.randn(batch_size, latent_dim).to(device)
                fake_imgs = generator(z).view(-1, 1, 28, 28)
                
                # Wasserstein 損失
                c_real = critic(real_imgs).mean()
                c_fake = critic(fake_imgs.detach()).mean()
                
                # 梯度懲罰
                gp = compute_gradient_penalty(critic, real_imgs, fake_imgs)
                
                # 評論器損失 = 假 - 真 + 懲罰
                c_loss = c_fake - c_real + lambda_gp * gp
                
                c_loss.backward()
                opt_C.step()
            
            # 訓練生成器
            opt_G.zero_grad()
            
            z = torch.randn(batch_size, latent_dim).to(device)
            fake_imgs = generator(z).view(-1, 1, 28, 28)
            
            # 生成器損失 = -假分數（想最大化假分數）
            g_loss = -critic(fake_imgs).mean()
            
            g_loss.backward()
            opt_G.step()
            
            c_losses.append(c_loss.item())
            g_losses.append(g_loss.item())
        
        history['c_loss'].append(np.mean(c_losses))
        history['g_loss'].append(np.mean(g_losses))
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}")
            print(f"  C Loss: {history['c_loss'][-1]:.4f}, G Loss: {history['g_loss'][-1]:.4f}")
            
            generator.eval()
            with torch.no_grad():
                samples = generator(fixed_noise).view(-1, 1, 28, 28)
            show_images(samples, nrow=8, title=f"WGAN-GP Epoch {epoch+1}")
            generator.train()
    
    return history

# 訓練 WGAN-GP
wgan_G = Generator(latent_dim=100).to(device)
wgan_C = WGANCritic().to(device)

wgan_history = train_wgan_gp(wgan_G, wgan_C, train_loader, epochs=50)

### 練習 3: 條件 Diffusion

**目標**: 為 Diffusion 模型添加類別條件

In [ ]:
# 練習 3: 條件 Diffusion

class ConditionalUNet(nn.Module):
    """
    帶類別條件的 U-Net
    """
    
    def __init__(self, in_channels=1, num_classes=10, time_dim=256, features=64):
        super().__init__()
        
        self.time_mlp = nn.Sequential(
            SinusoidalPositionEmbedding(time_dim),
            nn.Linear(time_dim, time_dim),
            nn.GELU(),
            nn.Linear(time_dim, time_dim),
        )
        
        # 類別嵌入
        self.class_embedding = nn.Embedding(num_classes, time_dim)
        
        # 下採樣
        self.down1 = self._conv_block(in_channels, features)
        self.down2 = self._conv_block(features, features * 2)
        self.pool = nn.MaxPool2d(2)
        
        # 中間
        self.mid = self._conv_block(features * 2, features * 2)
        
        # 條件投影（時間 + 類別）
        self.cond_proj = nn.Linear(time_dim * 2, features * 2)
        
        # 上採樣
        self.up1 = nn.ConvTranspose2d(features * 2, features * 2, 2, 2)
        self.up_conv1 = self._conv_block(features * 4, features)
        self.up2 = nn.ConvTranspose2d(features, features, 2, 2)
        self.up_conv2 = self._conv_block(features * 2, features)
        
        self.out = nn.Conv2d(features, in_channels, 1)
        
    def _conv_block(self, in_ch, out_ch):
        return nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.GroupNorm(8, out_ch),
            nn.GELU(),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.GroupNorm(8, out_ch),
            nn.GELU(),
        )
        
    def forward(self, x, t, y):
        """
        Args:
            x: 噪音圖像
            t: 時間步
            y: 類別標籤
        """
        # 時間和類別嵌入
        t_emb = self.time_mlp(t.float())
        c_emb = self.class_embedding(y)
        cond = torch.cat([t_emb, c_emb], dim=1)  # 合併條件
        
        # 下採樣
        d1 = self.down1(x)
        d2 = self.down2(self.pool(d1))
        
        # 中間（加入條件）
        m = self.mid(self.pool(d2))
        cond_proj = self.cond_proj(cond)[:, :, None, None]
        m = m + cond_proj
        
        # 上採樣
        u1 = self.up1(m)
        u1 = torch.cat([u1, d2], dim=1)
        u1 = self.up_conv1(u1)
        
        u2 = self.up2(u1)
        u2 = torch.cat([u2, d1], dim=1)
        u2 = self.up_conv2(u2)
        
        return self.out(u2)

def train_conditional_diffusion(model, scheduler, train_loader, epochs=20, lr=1e-4):
    """訓練條件 Diffusion"""
    
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    for epoch in range(epochs):
        model.train()
        losses = []
        
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
        
        for images, labels in pbar:
            images = images.to(device)
            labels = labels.to(device)
            batch_size = images.size(0)
            
            t = torch.randint(0, scheduler.num_timesteps, (batch_size,)).to(device)
            
            noise = torch.randn_like(images)
            noisy_images, _ = scheduler.add_noise(images, t.cpu(), noise)
            noisy_images = noisy_images.to(device)
            
            predicted_noise = model(noisy_images, t, labels)
            
            loss = F.mse_loss(predicted_noise, noise)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            losses.append(loss.item())
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        print(f"Epoch {epoch+1}, Avg Loss: {np.mean(losses):.4f}")
    
    return model

# 訓練
cond_diffusion = ConditionalUNet(num_classes=10).to(device)
scheduler = DiffusionScheduler(num_timesteps=1000)

cond_diffusion = train_conditional_diffusion(cond_diffusion, scheduler, train_loader, epochs=20)

In [ ]:
@torch.no_grad()
def sample_conditional_diffusion(model, scheduler, labels, img_size=28):
    """
    條件生成
    """
    model.eval()
    n_samples = len(labels)
    labels = labels.to(device)
    
    x = torch.randn(n_samples, 1, img_size, img_size).to(device)
    
    for t in tqdm(reversed(range(scheduler.num_timesteps)), desc="Sampling", total=scheduler.num_timesteps):
        t_batch = torch.full((n_samples,), t, dtype=torch.long).to(device)
        
        predicted_noise = model(x, t_batch, labels)
        
        alpha = scheduler.alphas[t]
        alpha_cumprod = scheduler.alpha_cumprod[t]
        beta = scheduler.betas[t]
        
        coef1 = 1 / torch.sqrt(alpha)
        coef2 = beta / torch.sqrt(1 - alpha_cumprod)
        
        mean = coef1 * (x - coef2 * predicted_noise)
        
        if t > 0:
            noise = torch.randn_like(x)
            sigma = torch.sqrt(beta)
            x = mean + sigma * noise
        else:
            x = mean
    
    return x

# 生成每個數字
fig, axes = plt.subplots(10, 8, figsize=(10, 12))

for digit in range(10):
    labels = torch.full((8,), digit, dtype=torch.long)
    samples = sample_conditional_diffusion(cond_diffusion, scheduler, labels)
    
    for i in range(8):
        axes[digit, i].imshow(samples[i].cpu().squeeze(), cmap='gray')
        axes[digit, i].axis('off')
    axes[digit, 0].set_ylabel(str(digit), fontsize=12, rotation=0, labelpad=15)

plt.suptitle('條件 Diffusion 生成結果')
plt.tight_layout()
plt.show()

---
## Part 5: 進階技巧與實務應用

### 5.1 GAN 訓練技巧

GAN 訓練是出了名的困難。以下是一些實用技巧：

In [ ]:
# ========== GAN 訓練技巧 ==========

# 技巧 1: Label Smoothing - 讓判別器不要太自信
def get_smoothed_labels(batch_size, device, label_type='real', smoothing=0.1):
    """
    標籤平滑：
    - 真實標籤: 0.9 而不是 1.0
    - 假標籤: 0.1 而不是 0.0
    這防止判別器過於自信，有助於生成器學習
    """
    if label_type == 'real':
        return torch.full((batch_size, 1), 1.0 - smoothing, device=device)
    else:
        return torch.full((batch_size, 1), smoothing, device=device)

# 技巧 2: Two Time-scale Update Rule (TTUR)
# 判別器使用較高學習率，生成器使用較低學習率
def create_ttur_optimizers(generator, discriminator):
    """
    TTUR: 判別器 lr > 生成器 lr
    這有助於訓練穩定性
    """
    opt_G = torch.optim.Adam(generator.parameters(), lr=1e-4, betas=(0.5, 0.999))
    opt_D = torch.optim.Adam(discriminator.parameters(), lr=4e-4, betas=(0.5, 0.999))
    return opt_G, opt_D

# 技巧 3: Feature Matching Loss - 讓生成圖像在特徵空間更接近真實圖像
class DiscriminatorWithFeatures(nn.Module):
    """
    可以輸出中間特徵的判別器
    用於 Feature Matching Loss
    """
    
    def __init__(self, img_dim=784):
        super().__init__()
        
        self.layer1 = nn.Sequential(
            nn.Linear(img_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
        )
        
        self.layer2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
        )
        
        self.out = nn.Sequential(
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )
        
    def forward(self, img, return_features=False):
        x = img.view(img.size(0), -1)
        f1 = self.layer1(x)
        f2 = self.layer2(f1)
        out = self.out(f2)
        
        if return_features:
            return out, [f1, f2]
        return out

def feature_matching_loss(discriminator, real_imgs, fake_imgs):
    """
    Feature Matching Loss:
    讓生成圖像的判別器特徵接近真實圖像的特徵
    """
    _, real_features = discriminator(real_imgs, return_features=True)
    _, fake_features = discriminator(fake_imgs, return_features=True)
    
    loss = 0
    for real_f, fake_f in zip(real_features, fake_features):
        loss += F.mse_loss(fake_f.mean(0), real_f.mean(0).detach())
    
    return loss

# 技巧 4: 檢測模式崩潰 (Mode Collapse)
def check_mode_collapse(generator, latent_dim=100, n_samples=100, threshold=0.1):
    """
    模式崩潰檢測：
    如果生成的圖像太相似，可能發生了模式崩潰
    """
    generator.eval()
    with torch.no_grad():
        z = torch.randn(n_samples, latent_dim).to(device)
        samples = generator(z).view(n_samples, -1)
        
        # 計算樣本間的平均距離
        distances = []
        for i in range(n_samples):
            for j in range(i+1, min(i+10, n_samples)):  # 只計算部分對
                dist = torch.dist(samples[i], samples[j]).item()
                distances.append(dist)
        
        avg_distance = np.mean(distances)
        std_distance = np.std(distances)
        
        print(f"樣本間平均距離: {avg_distance:.4f} ± {std_distance:.4f}")
        
        if avg_distance < threshold:
            print("⚠️ 警告：可能發生了模式崩潰！生成的樣本過於相似。")
            return True
        else:
            print("✓ 生成樣本具有足夠的多樣性")
            return False
    
    generator.train()

# 測試模式崩潰檢測
print("檢測 DCGAN 的模式崩潰情況：")
check_mode_collapse(dc_G)

### 5.2 DDIM: 加速 Diffusion 採樣

DDPM 需要 1000 步才能生成，太慢了！DDIM (Denoising Diffusion Implicit Models) 可以大幅減少步數。

```
DDPM: 1000 步 → 約 1000 次模型推理
DDIM: 50-100 步 → 相同品質，快 10-20 倍！

關鍵區別：
- DDPM: 隨機採樣（每步加噪音）
- DDIM: 確定性採樣（可以跳過步驟）
```

In [ ]:
# ========== DDIM 快速採樣 ==========

@torch.no_grad()
def ddim_sample(model, scheduler, n_samples=16, img_size=28, 
                ddim_steps=50, eta=0.0):
    """
    DDIM 採樣 - 大幅減少生成步數
    
    Args:
        ddim_steps: 實際採樣步數（可以遠小於訓練時的 1000 步）
        eta: 0 = 完全確定性, 1 = 等同 DDPM
    """
    model.eval()
    
    # 選擇子序列時間步 (均勻分布)
    # 例如: 1000 步中選 50 步 -> [0, 20, 40, 60, ..., 980]
    timesteps = np.linspace(0, scheduler.num_timesteps - 1, ddim_steps, dtype=int)
    timesteps = list(reversed(timesteps))  # 從高到低
    
    # 從純噪音開始
    x = torch.randn(n_samples, 1, img_size, img_size).to(device)
    
    for i, t in enumerate(tqdm(timesteps, desc="DDIM Sampling")):
        t_batch = torch.full((n_samples,), t, dtype=torch.long).to(device)
        
        # 預測噪音
        predicted_noise = model(x, t_batch)
        
        # 當前和下一個時間步的 alpha
        alpha_cumprod_t = scheduler.alpha_cumprod[t]
        
        if i < len(timesteps) - 1:
            alpha_cumprod_prev = scheduler.alpha_cumprod[timesteps[i + 1]]
        else:
            alpha_cumprod_prev = torch.tensor(1.0)
        
        # 預測 x0
        x0_pred = (x - torch.sqrt(1 - alpha_cumprod_t) * predicted_noise) / torch.sqrt(alpha_cumprod_t)
        x0_pred = torch.clamp(x0_pred, -1, 1)  # 穩定性
        
        # DDIM 更新公式
        sigma = eta * torch.sqrt((1 - alpha_cumprod_prev) / (1 - alpha_cumprod_t)) * \
                torch.sqrt(1 - alpha_cumprod_t / alpha_cumprod_prev)
        
        # "方向" 指向 x_t
        dir_xt = torch.sqrt(1 - alpha_cumprod_prev - sigma ** 2) * predicted_noise
        
        # 下一步
        x = torch.sqrt(alpha_cumprod_prev) * x0_pred + dir_xt
        
        # 如果 eta > 0, 加入一些隨機噪音
        if eta > 0 and i < len(timesteps) - 1:
            noise = torch.randn_like(x)
            x = x + sigma * noise
    
    return x

# 比較 DDPM vs DDIM 速度
import time

print("比較生成速度：")

# DDPM (原始方法)
start = time.time()
samples_ddpm = sample_diffusion(diffusion_model, scheduler, n_samples=4)
ddpm_time = time.time() - start
print(f"DDPM (1000 步): {ddpm_time:.2f} 秒")

# DDIM (50 步)
start = time.time()
samples_ddim_50 = ddim_sample(diffusion_model, scheduler, n_samples=4, ddim_steps=50)
ddim_50_time = time.time() - start
print(f"DDIM (50 步): {ddim_50_time:.2f} 秒")

# DDIM (100 步)
start = time.time()
samples_ddim_100 = ddim_sample(diffusion_model, scheduler, n_samples=4, ddim_steps=100)
ddim_100_time = time.time() - start
print(f"DDIM (100 步): {ddim_100_time:.2f} 秒")

print(f"\n加速比: DDPM vs DDIM-50 = {ddpm_time/ddim_50_time:.1f}x")

# 可視化比較
fig, axes = plt.subplots(3, 4, figsize=(8, 6))
for i in range(4):
    axes[0, i].imshow(samples_ddpm[i].cpu().squeeze(), cmap='gray')
    axes[0, i].axis('off')
    axes[1, i].imshow(samples_ddim_50[i].cpu().squeeze(), cmap='gray')
    axes[1, i].axis('off')
    axes[2, i].imshow(samples_ddim_100[i].cpu().squeeze(), cmap='gray')
    axes[2, i].axis('off')

axes[0, 0].set_ylabel('DDPM\n(1000步)', fontsize=10, rotation=0, labelpad=40)
axes[1, 0].set_ylabel('DDIM\n(50步)', fontsize=10, rotation=0, labelpad=40)
axes[2, 0].set_ylabel('DDIM\n(100步)', fontsize=10, rotation=0, labelpad=40)

plt.suptitle('DDPM vs DDIM 生成品質比較')
plt.tight_layout()
plt.show()

### 5.3 Classifier-Free Guidance (CFG)

Classifier-Free Guidance 是 Stable Diffusion、DALL-E 2 等模型的核心技術，讓你可以控制「條件」的強度。

```
概念：
- 訓練時：隨機丟棄條件（比如 10% 機率不提供類別標籤）
- 推理時：同時預測有條件和無條件的噪音，然後組合

CFG 公式：
noise_pred = noise_uncond + guidance_scale × (noise_cond - noise_uncond)

guidance_scale:
- 1.0 = 正常採樣
- > 1.0 = 更強的條件引導（更像指定的類別）
- >> 7.5 通常效果最好（Stable Diffusion 默認值）
```

In [ ]:
# ========== Classifier-Free Guidance ==========

class CFGConditionalUNet(nn.Module):
    """
    支援 Classifier-Free Guidance 的 U-Net
    
    關鍵：支持無條件生成（class = null）
    """
    
    def __init__(self, in_channels=1, num_classes=10, time_dim=256, features=64):
        super().__init__()
        
        self.num_classes = num_classes
        
        self.time_mlp = nn.Sequential(
            SinusoidalPositionEmbedding(time_dim),
            nn.Linear(time_dim, time_dim),
            nn.GELU(),
            nn.Linear(time_dim, time_dim),
        )
        
        # 類別嵌入（+1 給 "無條件" 類別）
        self.class_embedding = nn.Embedding(num_classes + 1, time_dim)
        
        # 網路結構同前
        self.down1 = self._conv_block(in_channels, features)
        self.down2 = self._conv_block(features, features * 2)
        self.pool = nn.MaxPool2d(2)
        
        self.mid = self._conv_block(features * 2, features * 2)
        self.cond_proj = nn.Linear(time_dim * 2, features * 2)
        
        self.up1 = nn.ConvTranspose2d(features * 2, features * 2, 2, 2)
        self.up_conv1 = self._conv_block(features * 4, features)
        self.up2 = nn.ConvTranspose2d(features, features, 2, 2)
        self.up_conv2 = self._conv_block(features * 2, features)
        
        self.out = nn.Conv2d(features, in_channels, 1)
        
    def _conv_block(self, in_ch, out_ch):
        return nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.GroupNorm(8, out_ch),
            nn.GELU(),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.GroupNorm(8, out_ch),
            nn.GELU(),
        )
        
    def forward(self, x, t, y):
        """
        y: 類別標籤，num_classes 表示無條件
        """
        t_emb = self.time_mlp(t.float())
        c_emb = self.class_embedding(y)
        cond = torch.cat([t_emb, c_emb], dim=1)
        
        d1 = self.down1(x)
        d2 = self.down2(self.pool(d1))
        
        m = self.mid(self.pool(d2))
        cond_proj = self.cond_proj(cond)[:, :, None, None]
        m = m + cond_proj
        
        u1 = self.up1(m)
        u1 = torch.cat([u1, d2], dim=1)
        u1 = self.up_conv1(u1)
        
        u2 = self.up2(u1)
        u2 = torch.cat([u2, d1], dim=1)
        u2 = self.up_conv2(u2)
        
        return self.out(u2)

def train_cfg_diffusion(model, scheduler, train_loader, epochs=15, lr=1e-4, 
                        p_uncond=0.1):
    """
    訓練 CFG Diffusion
    
    Args:
        p_uncond: 丟棄條件的機率（訓練無條件生成）
    """
    
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    num_classes = model.num_classes
    
    for epoch in range(epochs):
        model.train()
        losses = []
        
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
        
        for images, labels in pbar:
            images = images.to(device)
            labels = labels.to(device)
            batch_size = images.size(0)
            
            # 隨機丟棄條件
            # 用 num_classes 表示 "無條件"
            drop_mask = torch.rand(batch_size) < p_uncond
            labels = torch.where(
                drop_mask.to(device),
                torch.full_like(labels, num_classes),  # 無條件標籤
                labels
            )
            
            t = torch.randint(0, scheduler.num_timesteps, (batch_size,)).to(device)
            
            noise = torch.randn_like(images)
            noisy_images, _ = scheduler.add_noise(images, t.cpu(), noise)
            noisy_images = noisy_images.to(device)
            
            predicted_noise = model(noisy_images, t, labels)
            
            loss = F.mse_loss(predicted_noise, noise)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            losses.append(loss.item())
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})
        
        print(f"Epoch {epoch+1}, Avg Loss: {np.mean(losses):.4f}")
    
    return model

# 訓練
cfg_model = CFGConditionalUNet(num_classes=10).to(device)
cfg_model = train_cfg_diffusion(cfg_model, scheduler, train_loader, epochs=15)

---
## 🎯 總結：GAN 與 Diffusion 完整知識框架

### 📊 本模組重點回顧

| 主題 | 核心概念 | 關鍵公式/技巧 |
|------|----------|---------------|
| **GAN 基礎** | 對抗訓練 (G vs D) | $\min_G \max_D V(D,G)$ |
| **DCGAN** | 卷積架構 | 轉置卷積上採樣、步進卷積下採樣 |
| **CGAN** | 條件生成 | 類別嵌入 + 拼接 |
| **WGAN-GP** | 穩定訓練 | Wasserstein 距離 + 梯度懲罰 |
| **DDPM** | 去噪擴散 | $L = \|\epsilon - \epsilon_\theta(x_t, t)\|^2$ |
| **DDIM** | 快速採樣 | 確定性跳步採樣 |
| **CFG** | 引導強度 | $\epsilon = \epsilon_{uncond} + s(\epsilon_{cond} - \epsilon_{uncond})$ |

### 🔄 GAN vs Diffusion 深度比較

```
┌────────────────┬────────────────────────┬────────────────────────┐
│ 維度           │ GAN                    │ Diffusion              │
├────────────────┼────────────────────────┼────────────────────────┤
│ 訓練方式       │ 對抗遊戲（minimax）    │ 去噪預測（MSE）        │
│ 訓練穩定性     │ 困難（模式崩潰）       │ 穩定                   │
│ 生成速度       │ 快（1次前向）          │ 慢（多步迭代）         │
│ 圖像品質       │ 好                     │ 非常好                 │
│ 多樣性         │ 可能較低               │ 高                     │
│ 條件控制       │ CGAN                   │ CFG                    │
│ 理論基礎       │ 博弈論                 │ 得分匹配/機率論        │
│ 代表應用       │ StyleGAN, BigGAN       │ Stable Diffusion, DALL-E │
│ 採樣加速       │ N/A                    │ DDIM, DPM-Solver       │
└────────────────┴────────────────────────┴────────────────────────┘
```

### 🛠️ GAN 訓練問題診斷

```
問題 1: 模式崩潰 (Mode Collapse)
  症狀：生成的樣本都很相似
  解決：
    - WGAN-GP（梯度懲罰）
    - Minibatch Discrimination
    - Feature Matching Loss
    - Unrolled GAN

問題 2: 訓練不穩定 / 梯度消失
  症狀：D 太強，G 學不動
  解決：
    - Label Smoothing
    - TTUR（D 學習率 > G）
    - Spectral Normalization
    - 減少 D 訓練次數

問題 3: 生成品質差
  症狀：圖像模糊、有偽影
  解決：
    - 更深的網路
    - Progressive Growing
    - 更大 batch size
    - Self-Attention
```

### 🚀 Diffusion 模型演進路線

```
DDPM (2020)
  │  ↓ 加速採樣
DDIM (2020)
  │  ↓ 條件控制
Classifier Guidance (2021)
  │  ↓ 無需分類器
Classifier-Free Guidance (2022)
  │  ↓ 文字條件
Latent Diffusion / Stable Diffusion (2022)
  │  ↓ 更快採樣
DPM-Solver, Consistency Models (2023)
  │  ↓ 視頻生成
Sora, SVD (2024)
```

### 📝 實戰 Checklist

**GAN 訓練前**：
- [ ] 數據預處理：歸一化到 [-1, 1]
- [ ] 使用 LeakyReLU（不用 ReLU）
- [ ] G 輸出用 Tanh，D 輸出用 Sigmoid（BCE）或無（WGAN）
- [ ] 使用 BatchNorm（G）但不在 D 的第一層用

**GAN 訓練中**：
- [ ] 監控 D 和 G 的 loss 比例
- [ ] 定期生成樣本檢查品質
- [ ] 檢測模式崩潰（樣本多樣性）

**Diffusion 訓練前**：
- [ ] 選擇合適的 noise schedule（linear, cosine）
- [ ] U-Net 結構設計（足夠深度）
- [ ] 時間編碼選擇（正弦/學習）

**Diffusion 推理**：
- [ ] 選擇採樣器（DDPM, DDIM, DPM-Solver）
- [ ] 設定 CFG scale（通常 5-15）
- [ ] 調整採樣步數（quality vs speed）

### 🔗 延伸學習資源

| 資源 | 說明 |
|------|------|
| [GAN 原論文](https://arxiv.org/abs/1406.2661) | Goodfellow et al., 2014 |
| [DDPM 論文](https://arxiv.org/abs/2006.11239) | Ho et al., 2020 |
| [DDIM 論文](https://arxiv.org/abs/2010.02502) | Song et al., 2020 |
| [CFG 論文](https://arxiv.org/abs/2207.12598) | Ho & Salimans, 2022 |
| [Stable Diffusion](https://arxiv.org/abs/2112.10752) | Rombach et al., 2022 |
| [Lilian Weng's Blog](https://lilianweng.github.io/) | 深度學習科普 |

### ⏭️ 下一步

在下一個模組中，我們將學習 **圖神經網路 (GNN)**：
- 處理圖結構資料（社交網路、分子、知識圖譜）
- Graph Convolutional Networks (GCN)
- Graph Attention Networks (GAT)
- 消息傳遞神經網路 (MPNN)

---
## 總結

### 本模組重點

1. **GAN 基礎**
   - 對抗訓練：生成器 vs 判別器
   - 最小-最大遊戲目標
   - DCGAN 使用卷積提升品質

2. **GAN 變體**
   - CGAN：條件生成
   - WGAN-GP：更穩定的訓練

3. **Diffusion 模型**
   - 前向：逐步加噪
   - 反向：學習去噪
   - U-Net 預測噪音

### GAN vs Diffusion 比較

```
┌────────────┬──────────────────┬──────────────────┐
│ 特性       │ GAN              │ Diffusion        │
├────────────┼──────────────────┼──────────────────┤
│ 訓練穩定性 │ 較難（模式崩潰） │ 較穩定           │
│ 生成速度   │ 快（一次前向）   │ 慢（需多步迭代） │
│ 圖像品質   │ 好               │ 非常好           │
│ 多樣性     │ 可能較低         │ 較高             │
│ 理論基礎   │ 博弈論           │ 機率論           │
└────────────┴──────────────────┴──────────────────┘
```

### 下一步

在下一個模組中，我們將學習 **圖神經網路 (GNN)**，處理圖結構資料。